In [61]:
filename = "ld_st_test/ld_st_test"

with open(f"{filename}_data.txt") as fp:
    b = fp.read().split("\n")

data = []
data_counter = 0
labels = {}

for row in b:
    row = row.strip()                   # Remove whitespace
    if row == "" or row[0:2] == "||":   # If line is empty / comment line, ignore
        continue
    elif row[-1] == ":":
        labels[row[:-1]] = data_counter
    else:
        row = row.strip().split("||")
        data.append(row[0].strip())
        print(f"{data_counter} : {data[-1]}")
        data_counter += 2

with open(f"{filename}_data.uasm", "w+") as fp:
    fp.write("\n".join(data[::-1]))         # Write in reverse since const indexing in Lucid starts from the right

with open(f"{filename}_data.uasm", "w+") as fp:
    fp.write(",\n".join([f"b{i}" for i in data[::-1]]))

print(f"Number of data memory entries: {data_counter // 2}")
# print(labels)

0 : 0000000000000001
2 : 0000000000000101
4 : 0000000000010101
6 : 0000000000001010
8 : 0000000000001111
10 : 0000000000011111
Number of data memory entries: 6


In [62]:
with open(f"{filename}.txt") as fp:
    a = fp.read().split("\n")

# Initial cleaning up
parsed_rows = []
for row in a:
    row = row.strip()                   # Remove whitespace
    if row == "" or row[0:2] == "||":   # If line is empty / comment line, ignore
        continue
    else:
        row = row.strip().split("||")   # Else, split on comment and take first element (ASM instruction) without whitespace
        parsed_rows.append(row[0].strip())

with open("1D pseudocode no comments.uasm", "w+") as fp:
    data = "\n".join(parsed_rows)
    fp.write(data)

one_x_zero = {
    "ADD" : "0000",
    "SUB" : "0001",
    "CMPEQ" : "0100",
    "CMPLT" : "0101",
    "CMPLE" : "0110",
}

one_x_one = {
    "AND" : "1000",
    "OR" : "1001",
    "XOR" : "1010",
    "SHL" : "1100",
    "SHR" : "1101",
    "SRA" : "1110"
}

op_lookup = {
    "RAND" : rand,
    "BEQ" : beq,
    "BNE" : bne,
    "JMP" : jmp,
    "LD" : ld,
    "ST" : st,
    "NOP" : nop,
    "SVC" : svc,
    "MOVE" : move,
    "CMOVE": cmove,
    "BF": beq, 
    "BT": bne,
    "BR": br,
    "PUSH": push,
    "POP": pop,
    "ALLOCATE": allocate,
    "DEALLOCATE": deallocate
}

reg_lookup = {
    "BP" : "R27",
    "LP" : "R28",
    "SP" : "R29",
    "XP" : "R30",
}

instruction_counter = 0
instructions = []

subbed_asm = []

def parse_op_str(s):
    # print(s)
    if not s.isnumeric():
        op, args = s[:-1].split("(")
        if len(args) > 0:
            args = [parse_op_arg(i.strip(), op.lower() in ["push", "pop", "ld", "st"]) for i in args.split(",")]
        else:
            args = []
        
        # print(f"Op: {op}")
        # print(f"Args: {args}")

        if op in one_x_zero:
            instructions.append(arith("10" + one_x_zero[op], args))
        elif op[:-1] in one_x_zero:
            instructions.append(arith("11" + one_x_zero[op[:-1]], args))
        elif op in one_x_one:
            instructions.append(arith("10" + one_x_one[op], args))
        elif op[:-1] in one_x_one:
            instructions.append(arith("11" + one_x_one[op[:-1]], args))
        elif op in op_lookup:
            instructions.append(op_lookup[op](args))
        else:
            raise Exception(f"Non-op string {s} provided.")
    else:
        instructions.append(s)
    # print(instructions[-1])
    # print()

def parse_op_arg(s, mem=False):
    if s in reg_lookup:
        s = reg_lookup[s]
    if s[0].lower() == "r":
        s = s[1:]
        return bin(int(s))[2:].zfill(5)
    elif s.isnumeric():
        # We are reading a literal:
        return bin(int(s))[2:].zfill(16)
    elif s[1:].isnumeric():
        return dec_to_tcbin(int(s))
    elif s in labels:
        if not mem:
            # Have to convert from label to literal
            # print(labels[s])
            literal = ((labels[s] - instruction_counter) // 4) - 1
            # print(literal)
            return dec_to_tcbin(literal)
        else:
            return dec_to_tcbin(labels[s])

    else:
        raise Exception()

def dec_to_tcbin(x):
    if x < 0:
        x = (abs(x) ^ 131071) + 1
    return bin(x)[2:].zfill(16)[-16:]


for idx, row in enumerate(parsed_rows):
    if row[-1] == ":":
        # Add to our labels table and set it to current instruction counter value
        labels[row[:-1]] = instruction_counter
    elif row[0] == ".":
        address = row.split("=")[1].strip()
        while instruction_counter < int(address) * 4:
            instruction_counter += 4
    else:
        if row.split("(")[0].lower() in ["ld", "st"]:
            instruction_counter += 4
        elif row.split("(")[0].lower() in ["push", "pop"]:
            instruction_counter += 8
        instruction_counter += 4

print(labels)

instruction_counter = 0

for idx, row in enumerate(parsed_rows):
    if row[0] == ".":
        address = row.split("=")[1].strip()
        while instruction_counter < int(address) * 4:
            instructions.append("0" * 32)
            instruction_counter += 4
    elif row[-1] != ":":
        try:
            parse_op_str(row)
        except Exception as e:
            print(row)
            raise e
        
        print(f"{instruction_counter:<10} : {row:<50} : {instructions[-1]}")
        instruction_counter += 4 * (len(instructions[-1]) // 32)
    


with open(f"{filename}.uasm", "w+") as fp:
    fp.write("\n".join(instructions[::-1]))

with open(f"{filename}.uasm") as fp:
    data = fp.read().split("\n")

with open(f"{filename}_fpga.uasm", "w+") as fp2:
    data = [f"b{i}" for i in data]
    fp2.write(",\n".join(data))




{}
0          : LD(0, R24)                                         : 01100011000111110000000000000000
01100011000111110000000000000000
8          : LD(1, R24)                                         : 01100011000111110000000000000001
01100011000111110000000000000001
16         : LD(2, R24)                                         : 01100011000111110000000000000010
01100011000111110000000000000010
24         : LD(3, R24)                                         : 01100011000111110000000000000011
01100011000111110000000000000011
32         : LD(4, R24)                                         : 01100011000111110000000000000100
01100011000111110000000000000100
40         : LD(5, R25)                                         : 01100011001111110000000000000101
01100011001111110000000000000101
48         : BEQ(R31, 0, R31)                                   : 01110111111111110000000000000000


In [59]:
def ld(args):
    if len(args) == 2:
        args.insert(0, "11111")
    return f"011000{args[2]}{args[0]}{args[1]}\n011000{args[2]}{args[0]}{args[1]}"

def st(args):
    if len(args) == 2:
        args.append("11111")
    return f"011001{args[0]}{args[2]}{args[1]}\n011001{args[0]}{args[2]}{args[1]}"

def rand(args):
    return f"100111{args[0]}" + "0" * 21

def beq(args):
    if len(args) == 2:
        args.append("11111")
    return f"011101{args[2]}{args[0]}{args[1]}"

def bne(args):
    if len(args) == 2:
        args.append("11111")
    return f"011110{args[2]}{args[0]}{args[1]}"

def jmp(args):
    if len(args) == 1:
        args.append("11111")
    return f"011011{args[1]}{args[0]}" + "0" * 16

def nop(args):
    return "011111" + "0" * 26

def svc(args):
    return "011010" + "0" * 26

def move(args):
    temp_args = args
    temp_args.insert(1, "11111")
    return arith("100000", temp_args)

def cmove(args):
    temp_args = args
    temp_args.insert(0, "11111")
    return arith("110000", temp_args)

def br(args):
    if len(args) == 1:
        return beq(["11111", *args, "11111"])
    else:
        return beq(["11111", *args])

def push(args):
    sp = "11101"
    four = "0000000000000100"
    neg_four = "1111111111111100"
    addc_instr = arith("110000", [sp, four, sp])
    st_instr = st([*args, neg_four, "11101"])
    return f"{addc_instr}\n{st_instr}"

def pop(args):
    sp = "11101"
    four = "0000000000000100"
    neg_four = "1111111111111100"
    ld_instr = ld([sp, neg_four, *args])
    subc_instr = arith("110001", [sp, four, sp])
    return f"{ld_instr}\n{subc_instr}"

def allocate(args):
    sp = "11101"
    alloc_val = parse_op_arg(str(4 * int(args[0])))
    return arith("110000", [sp, alloc_val, sp])

def deallocate(args):
    sp = "11101"
    alloc_val = parse_op_arg(str(4 * int(args[0])))
    return arith("110001", [sp, alloc_val, sp])

def arith(opcode, args):
    return f"{opcode}{args[2]}{args[0]}{args[1]}" + ("0" * 11 * (int(opcode[1]) == 0))

In [67]:
data = []

for i in range(500):
    data.append(f"b00000000000000000000000000000000,")
with open("reg_temp.txt", "w+") as fp:   
    fp.write("\n".join(data))